# Meta-analysis of antidepressant exposure GWAS

Conduct GWAS meta-analysis using [Hail](https://hail.is). Import GWAS sumstats in [GWAS VCF](https://github.com/MRCIEU/gwas-vcf-specification) format, which can be read into Hail and understood like any other VCF file. See also [meta-analysis functions from the Pan UK Biobank](https://github.com/atgu/ukbb_pan_ancestry/blob/master/utils/meta_analysis.py). 

In [1]:
import pandas as pd
import hail as hl
hl.init(local='local[8]')

23/07/11 10:57:52 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/07/11 10:57:52 WARN Hail: This Hail JAR was compiled for Spark 3.3.0, running with Spark 3.3.2.
  Compatibility is not guaranteed.


Running on Apache Spark version 3.3.2
SparkUI available at http://sakyo.home:4040
Welcome to
     __  __     <>__
    / /_/ /__  __/ /
   / __  / _ `/ / /
  /_/ /_/\_,_/_/_/   version 0.2.112-31ceff2fb5fd
LOGGING: writing to /Users/mark/Work/antidep-gwas/hail-20230711-1057-0.2.112-31ceff2fb5fd.log


Import GWAS VCFs and convert to `MatrixTable`s.

In [2]:
from os import path
import os
import glob

if not os.path.exists("mt"):
    os.makedirs("mt")

for gwas_vcf in glob.glob("vcf/*.vcf.gz"):
    dataset = path.basename(gwas_vcf).split(os.extsep)[0]
    gwas_mt = path.join("mt", os.extsep.join([dataset, "mt"]))
    if not path.exists(gwas_mt):
        hl.import_vcf(gwas_vcf,  reference_genome="GRCh38", force_bgz=True).write(gwas_mt, overwrite=True)


Open all the sumstats tables

In [3]:
mts = [hl.read_matrix_table(mt) for mt in glob.glob("mt/*.mt")]

Merge the list of sumstats together.

In [4]:
import functools

gw = functools.reduce(lambda mt1, mt2: mt1.union_cols(mt2, row_join_type = "outer"), mts)

In [85]:
list(gw.entry)

['ES', 'SE', 'LP', 'AF', 'SS', 'EZ', 'SI', 'NC', 'ID']

Get sumstats meta information

In [26]:
meta = hl.import_table("sumstats.csv", delimiter=",", key="cohort")

Annotate sumstats columns with cluster group

In [25]:
gw = gw.annotate_cols(cluster=meta[gw.s].cluster)
gw.cluster.show()

,
s,cluster
str,str
"""BBJ""","""EAS"""
"""GenScot""","""EUR"""
"""FinnGen""","""EUR"""
"""UKB""","""EUR"""


Meta-analysis weight for each sumstats entry (IVW)

In [87]:
# inverse variance weight for each estimate
mw = gw.annotate_entries(
    ivw=1 / gw.SE ** 2
)

# effect weighted by inverse variance
mw = mw.annotate_entries(
    ES_w=mw.ES * mw.ivw
)

Meta-analysis aggregation for each cluster

In [88]:
ma = mw.group_cols_by(mw.cluster).aggregate(
    ES=hl.agg.array_sum(mw.ES_w) / hl.agg.array_sum(mw.ivw),
    SE=hl.map(lambda ivw_sum: hl.sqrt(1 / ivw_sum), hl.agg.array_sum(mw.ivw)),
    NC=hl.agg.array_sum(mw.NC),
    SS=hl.agg.array_sum(mw.SS)
)

In [89]:
ma.ES.show()

,,,
,,'EAS','EUR'
locus,alleles,ES,ES
locus<GRCh38>,array<str>,array<float64>,array<float64>
chr1:10177,"[""A"",""AC""]",NA,[-1.53e-02]
chr1:10511,"[""G"",""A""]",NA,[-1.88e-02]
chr1:10616,"[""C"",""CCGCCGTTGCAAAGGCGCGCCG""]",NA,[-6.06e-02]
chr1:11008,"[""C"",""G""]",NA,[-2.19e-03]
chr1:11012,"[""C"",""G""]",NA,[-2.19e-03]
chr1:13110,"[""G"",""A""]",NA,[-1.38e-02]
chr1:13116,"[""T"",""G""]",NA,[7.60e-03]
